# Dagstuhl ChoirSet: Evaluation of F0-Annotations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mir_eval

import os
import glob

import DCStoolbox as dcst

In [ ]:
tony_path = r'/Users/rosenzweig/Google Drive/Dagstuhl Dataset/Annotations/pitch/Tony/'
pyin_path =  '/Users/rosenzweig/Documents/data/DagstuhlChoirSet_V1.0/annotations_csv_F0_PYIN/'
crepe_path = '/Users/rosenzweig/Documents/data/DagstuhlChoirSet_V1.0/annotations_csv_F0_CREPE/'


In [ ]:
file_list = np.sort(glob.glob((pyin_path + '*.csv')))

eval_pyin_lrx = np.empty((0,5))
eval_pyin_hsm = np.empty((0,5))
eval_crepe_lrx = np.empty((0,5))
eval_crepe_hsm = np.empty((0,5))

for pyin_file in file_list:
    fn = os.path.basename(pyin_file)[:-4]
    
    anno_file = glob.glob(tony_path + fn[:-3] + '*_pitch.csv')
    if not anno_file:
        continue
    else:
        anno_file = anno_file[0]
    crepe_file = crepe_path + fn + '.csv'
    print(fn)
    
    # read files
    header = None
    names = None
    anno = pd.read_csv(anno_file, sep=',', keep_default_na=False, header=header, names=names).values
    pyin = pd.read_csv(pyin_file, sep=',', keep_default_na=False, header=header, names=names).values
    crepe = pd.read_csv(crepe_file, sep=',', keep_default_na=False, header=header, names=names).values
    
    # pad annotations with zeros
    pref = np.arange(0, anno[0, 0], anno[1, 0]-anno[0, 0]).reshape(-1,1)
    anno = np.vstack((np.hstack((pref, np.zeros(pref.shape))), anno))
    
    # remove discontinuities in time axis
    anno = np.delete(anno, np.where(anno[:-1, 0] >= anno[1:, 0])[0], axis=0)

    # resample to common time axis
    kind = 'cubic'
    t = np.arange(0, np.min([anno[-1, 0], pyin[-1, 0], crepe[-1, 0]]), 0.01)    
    anno_res, anno_va = mir_eval.melody.resample_melody_series(anno[:, 0], anno[:, 1], anno[:,1]>0, 
                                                               t, kind=kind)
    pyin_res, pyin_va = mir_eval.melody.resample_melody_series(pyin[:, 0], np.abs(pyin[:, 1]), pyin[:, 1]>0, 
                                                               t, kind=kind)
    crepe_res, crepe_va = mir_eval.melody.resample_melody_series(crepe[:, 0], crepe[:, 1], crepe[:, 2]>0.5, 
                                                                 t, kind=kind)
    
    # evaluate
    tol_cents = 20
    eval_pyin = mir_eval.melody.evaluate(t, anno_res, t, pyin_res*pyin_va, cent_tolerance=tol_cents)
    eval_crepe = mir_eval.melody.evaluate(t, anno_res, t, crepe_res*crepe_va, cent_tolerance=tol_cents)
    
    if "LRX" in fn:
        eval_pyin_lrx = np.append(eval_pyin_lrx, np.array([[eval_pyin['Voicing Recall'], 
                                                            eval_pyin['Voicing False Alarm'], 
                                                            eval_pyin['Raw Pitch Accuracy'], 
                                                            eval_pyin['Raw Chroma Accuracy'], 
                                                            eval_pyin['Overall Accuracy']]]), axis=0)
        eval_crepe_lrx = np.append(eval_crepe_lrx, np.array([[eval_crepe['Voicing Recall'], 
                                                            eval_crepe['Voicing False Alarm'], 
                                                            eval_crepe['Raw Pitch Accuracy'], 
                                                            eval_crepe['Raw Chroma Accuracy'], 
                                                            eval_crepe['Overall Accuracy']]]), axis=0)
    else:
        eval_pyin_hsm = np.append(eval_pyin_hsm, np.array([[eval_pyin['Voicing Recall'], 
                                                            eval_pyin['Voicing False Alarm'], 
                                                            eval_pyin['Raw Pitch Accuracy'], 
                                                            eval_pyin['Raw Chroma Accuracy'], 
                                                            eval_pyin['Overall Accuracy']]]), axis=0)
        eval_crepe_hsm = np.append(eval_crepe_hsm, np.array([[eval_crepe['Voicing Recall'], 
                                                            eval_crepe['Voicing False Alarm'], 
                                                            eval_crepe['Raw Pitch Accuracy'], 
                                                            eval_crepe['Raw Chroma Accuracy'], 
                                                            eval_crepe['Overall Accuracy']]]), axis=0)

In [ ]:
print('Voicing Recall, Voicing False Alarm, Raw Pitch Accuracy, Raw Chroma Accuracy, Overall Accuracy')
print('LRX (PYIN, CREPE)')
print(np.mean(eval_pyin_lrx, axis=0), np.mean(eval_crepe_lrx, axis=0))
print('HSM (PYIN, CREPE)')
print(np.mean(eval_pyin_hsm, axis=0), np.mean(eval_crepe_hsm, axis=0))

In [ ]:
print('HSM: Voicing Recall, Voicing False Alarm, Raw Pitch Accuracy, Raw Chroma Accuracy, Overall Accuracy')
np.mean(eval_pyin_hsm, axis=0), np.mean(eval_crepe_hsm, axis=0)

In [ ]:
%matplotlib notebook
plt.figure()
plt.plot(t, anno_res*anno_va, marker='.', linestyle='')
plt.plot(t, pyin_res*pyin_va, marker='.', linestyle='')
plt.plot(t, crepe_res*crepe_va, marker='.', linestyle='')
plt.legend(('anno', 'pyin', 'crepe'))